In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
import pandas as pd
import os
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base
# Import Matplot lib
import matplotlib
from matplotlib import style
style.use('seaborn')
import matplotlib.pyplot as plt

In [ ]:
# Store filepath in a variable
file_one = os.path.join("Resources", "cleaned_hawaii_measurements.csv")
file_two = os.path.join("Resources", "hawaii_stations.csv")
# Read our Data file with the pandas library
# Not every CSV requires an encoding, but be aware this can come up
hawaii_measurements_df = pd.read_csv(file_one, encoding="ISO-8859-1")
hawaii_stations_df = pd.read_csv(file_two, encoding="ISO-8859-1")

In [ ]:
# Use Orient='records' to create a list of data to write
# to_dict() cleans out DataFrame metadata as well
# http://pandas-docs.github.io/pandas-docs-travis/io.html#orient-options
station_data = hawaii_stations_df.to_dict(orient='records')
measurement_data = hawaii_measurements_df.to_dict(orient='records')

In [ ]:
# Create an engine to a SQLite database file called `customers.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [ ]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [ ]:
# Save the reference to the `customers` table as a variable called `table`
station_table = sqlalchemy.Table('Station', metadata, autoload=True)
measurement_table = sqlalchemy.Table('Measurement', metadata, autoload=True)

In [ ]:
# Use `table.delete()` to remove any pre-existing data.
# Note that this is a convenience function so that you can re-run the example code multiple times.
# You would not likely do this step in production.
conn.execute(station_table.delete())
conn.execute(measurement_table.delete())

In [ ]:
# Use `table.insert()` to insert the data into the table
# The SQL table is populated during this step
conn.execute(station_table.insert(), station_data)
conn.execute(measurement_table.insert(), measurement_data)

In [ ]:
# Test that the insert works by fetching . 
conn.execute("select * from Station ").fetchall()

In [ ]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from Measurement limit 5").fetchall()

In [ ]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)
Measurement = Base.classes.measurement

In [ ]:
# Start a session to query the database
session = Session(engine)

In [ ]:
# Query Emojis for `emoji_char`, `emoji_id`, and `score` and save the query into results
results = session.query(Measurement.date, Measurement.prcp).order_by(Measurement.date.desc()).all()

In [ ]:
len(results)

In [ ]:
# Load the results into a pandas dataframe. Set the index to the `emoji_id`
df = pd.DataFrame(results[:10], columns=['date', 'prcp'])
df.set_index('date', inplace=True, )
df.head(10)
